In [1]:
import cv2
import os
import math
import numpy as np
from PIL import Image
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
def auto_extract_frame(video_name, video_path, output_path, frame_rate, s_list) :

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("auto_extract_frame에서 비디오 파일을 열 수 없습니다.")
        return

    # FPS(Frame Per Second) 정보 가져오기
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))

    for i in range(len(s_list)) : # 접촉 시점마다 전후 1초동안 7장의 이미지를 추출

        start_time = s_list[i]
        interval = 2

        # 원하는 시간의 프레임 번호 계산
        frame_number = int((start_time- 1) * fps)
        end_frame_number = frame_number + interval * fps + 1
        
        while(frame_number <= end_frame_number) :
             
            # 해당 프레임으로 위치 설정
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
            # 프레임 읽기
            ret, frame = cap.read()
        
            if ret:
                # 프레임을 이미지 파일로 저장
                image_output_path = f"{output_path}/{video_name[:-4]}_startTime{start_time}_{frame_number}.jpg"
                cv2.imwrite(image_output_path, frame)
            else:
                print("프레임을 읽을 수 없습니다.")
                break
            
            frame_number += int(fps / frame_rate)
        
        i += 1

    # 비디오 캡처 객체 해제
    cap.release()


def touch_detection(video_name, video_path, output_path, start_time, frame_rate, model_path):

    print(f"{video_name}, 접촉 여부 판정이 시작되었습니다")
    
    # 모형 불러오기
    model = models.load_model(model_path) # model 불러오기

    # 비디오 파일 열기
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("touch_detection에서 비디오 파일을 열 수 없습니다.")
        return
    
    # FPS(Frame Per Second) 정보 가져오기
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS))
    
    if fps == 0:
        print("FPS 정보를 가져올 수 없습니다.")
        return
    
    # 원하는 시간의 프레임 번호 계산
    frame_number = int(start_time * fps)
    
    # frame_list = []
    touch_time = []
    while(True) : #동영상 끝까지 이미지를 추출
        
        # 해당 프레임으로 위치 설정
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # 프레임 읽기
        ret, frame = cap.read()
    
        if ret:
            # 프레임을 이미지 파일로 저장
            print(output_path)
            image_output_path = f"{output_path}/{video_name[:-4]}_{int(frame_number / fps)}.jpg"
            cv2.imwrite(image_output_path, frame)

            # 이미지 가공
            IMG_WIDTH, IMG_HEIGHT = 224, 224
            img = Image.open(image_output_path)
            img = img.resize((IMG_WIDTH, IMG_HEIGHT))
            np_img = image.img_to_array(img)
            img_batch = np.expand_dims(np_img, axis=0) #이미지 4차원으로 변형
            pre_processed = preprocess_input(img_batch) # nomalization

            # 모형을 통한 이미지의 접촉 여부 예측
            y_preds = model.predict(pre_processed)

            # 접촉 여부 확률에 따른 접촉 시간 저장
            if(y_preds[0][1] > 0.6) :

                end_frame_number = frame_number + fps 
                cap.set(cv2.CAP_PROP_POS_FRAMES, end_frame_number)
                ret, frame = cap.read()
                if ret != True :
                    os.remove(image_output_path)
                    break
                
                touch_time.append(int(frame_number / fps))
                # frame_list.append(frame_number)
                
                print(f"{int(frame_number / fps)}초에 접촉이 감지되었습니다.")
                print("접촉 일어났을 확률 :", y_preds[0][1])
            else :
                os.remove(image_output_path)
        else:
            print("프레임을 읽을 수 없습니다.")
            break

        frame_number += int(fps / frame_rate)
    
    # 비디오 캡처 객체 해제
    cap.release()
    return touch_time

In [3]:
def video_to_image(video_name, video_path) :
    
    model_path = "./model0_VGG16_layer5_RMSprop.keras"
    
    frame_rate_check_touch = 1  # 1초당 1장을 저장
    frame_rate_check_child_abuse = 3  # 1초당 3장
    
    output_path_check_touch = "./check_touch"
    output_path_check_child_abuse = "./check_child_abuse"
    start_time = 1
    
    # video_name = "test_video.mp4"
    # video_path = video_folder_path + "/" + video_name
        
    touch_times = touch_detection(video_name, video_path, output_path_check_touch, start_time, frame_rate_check_touch, model_path)
    print(touch_times)
    auto_extract_frame(video_name, video_path, output_path_check_child_abuse, frame_rate_check_child_abuse, touch_times)
        
    print("")
    print(f"{video_name}번째 영상에 대한 학대 의심 장면 추출이 완료되었습니다.")
    print("")

    return touch_times

In [4]:
# 각 sequece에서 원하는 만큼의 이미지를 추출
def data_maker(all_images, n) :

    selected_images = []
    if n == 7 :
        selected_images = all_images

    else :
        # 7장의 이미지 중 가운데 n장만 선택
        for i in range(0, len(all_images), 7):
            
            start = i + int((7 - n) / 2)
            end = i + int((7 + n) / 2)
           
            selected_images.extend(all_images[start:end])

    images = [image.load_img(p, target_size=(224, 224)) for p in selected_images]
    vector = np.asarray([image.img_to_array(img) for img in images])

    return vector

In [5]:
def delete_all_images_in_folder(folder_path):
    # 이미지 파일 경로 리스트 가져오기
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    
    for image_file in image_files:
        try:
            os.remove(os.path.join(folder_path, image_file))
            # print(f"Deleted {image_file}")
        except Exception as e:
            print(f"Error deleting {image_file}: {e}")

In [6]:
from glob import glob
import joblib
import shutil
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# check_times = len(touch_times)

def child_abuse_detection(video_name) :

    video_path = "./" + video_name
    touch_times = video_to_image(video_name, video_path)

    all_images = glob('./check_child_abuse/*jpg')
    
    images = all_images
    image_per_sequence = 3

    base_model = VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)
    
    vector = data_maker(images, image_per_sequence)
    vectors = model.predict(preprocess_input(vector))
    
    
    def concatenate_vectors(vectors, n):
        concatenated_vectors = []
        for i in range(0, vectors.shape[0], n):
            concatenated = vectors[i:i+n].reshape(1, -1)
            concatenated_vectors.append(concatenated)
        return np.vstack(concatenated_vectors)
    
    # 함수 호출
    concatenated_vectors = concatenate_vectors(vectors, image_per_sequence)
    
    # print(concatenated_vectors.shape)
    model2 = joblib.load('Model2_Seq3_VGG16.joblib')
    
    # 불러온 모델로 예측, 0 -> 학대X / 1 -> 학대O
    predictions = model2.predict(concatenated_vectors)
    # print(predictions)

    # 예측 결과에 따라 파일 이동
    output_folder = "./child_abuse_confirmed"

    ca_time = []
    for i in range(len(predictions)):
        if predictions[i] == 1:
            
            ca_time.append(touch_times[i])
            
            # 이동할 이미지 선택
            print(f"{video_name}의 {touch_times[i]}초에 학대가 의심되는 행동이 감지되어 이미지를 저장합니다")
            start_index = 7 * i
            end_index = start_index + 7
            selected_images = images[start_index:end_index]
            
            for image in selected_images:
                # 파일 이동
                shutil.move(image, os.path.join(output_folder, os.path.basename(image)))
                # print(f"Moved {image} to {output_folder}")

    if(len(ca_time) == 0) :
        print("학대 의심 행동이 발견되지 않았습니다")
    else :
        print("")
        print(f"총 {len(ca_time)}건의 학대 의심 행동이 발견되었습니다")
        print(f"영상 내 학대 발견된 지점(s) : {ca_time}")
        print("")

    delete_all_images_in_folder("./check_child_abuse")
    delete_all_images_in_folder("./check_touch")

In [7]:
child_abuse_detection("test_video1.mp4")

test_video1.mp4, 접촉 여부 판정이 시작되었습니다
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
1초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.6245402
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
9초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.71140313
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
10초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.66036457
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms

In [8]:
child_abuse_detection("test_video2.mp4")

test_video2.mp4, 접촉 여부 판정이 시작되었습니다
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
./check_tou

In [9]:
child_abuse_detection("test_video3.mp4")

test_video3.mp4, 접촉 여부 판정이 시작되었습니다
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 760ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
3초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9974841
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
4초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.99720746
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
5초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.99859494
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
6초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9991135
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
7초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9995995
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
8초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.99482006
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
9초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9971365
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
10초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.9988739
./check_touch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
11초에 접촉이 감지되었습니다.
접촉 일어났을 확률 : 0.99784935
./check_touch
1/